# 11 CMAP enrichment - for LK

cancer based egenes

2/10/2021


In [34]:
library(tidyverse)
# library(readr)
library(ggplot2)
library(clusterProfiler)
library(org.Hs.eg.db)
library(pheatmap)
library(ReactomePA)
library(annotate)
library(seqinr)

In [108]:
save_path = '../data/processed/drug_enrichment/'

In [10]:
genes_file = '../data/external/GWAS/cancer_mpra_hit_egenes_02092021.tsv'
genes_df = read.table(genes_file,header=T,stringsAsFactor=F)
head(genes_df)
diseases = sort(unique(genes_df$disease))
print(diseases)

,egene,disease
,<chr>,<chr>
1,A4GALT,hmec
2,AAGAB,thy
3,AAMP,colon
4,ABHD12,colon
5,ABHD8,hmec
6,ABHD8,ov


 [1] "airway" "ast"    "colon"  "endo"   "eso"    "gm"     "hmec"   "kc"    
 [9] "mc"     "ov"     "panc"   "pros"   "renal"  "thy"   


In [146]:
get_genes = function(dz, df=genes_df){
    print(dz)
    egenes = df[df$disease ==dz,'egene']
    print(length(egenes))
    return(egenes)
}

get_entrez = function(genes){
    entrez_ids = bitr(genes, fromType="SYMBOL", toType="ENTREZID", OrgDb="org.Hs.eg.db")$ENTREZID
#     print(length(entrez_ids))
    return(entrez_ids)
#     return(bitr(genes, fromType="SYMBOL", toType="ENTREZID", OrgDb="org.Hs.eg.db"))
}

enrichr_res = function(entrez_genes, c_gmt){
    enrichr_res <- enricher(entrez_genes, TERM2GENE=c_gmt,                                            
                                            minGSSize=0,
                                            maxGSSize=1000,                                                
                                            pAdjustMethod = "fdr",
                                            qvalueCutoff = 1,
                                            pvalueCutoff = 1)
    
    if (!is.null(enrichr_res)){
        enrichr_res <- setReadable(enrichr_res, OrgDb=org.Hs.eg.db, keyType="ENTREZID")
        return(data.frame(enrichr_res))}
    return(data.frame())
}

filt_res = function(df){
    return(filter(df,p.adjust<0.05))
}

get_lengths = function(list_dfs){
    for (name in names(list_dfs)){
        print(name)
        if (is.data.frame(list_dfs[[name]])){
            print(dim(list_dfs[[name]])[1])
        }
        else{
            print(length(list_dfs[[name]]))
        }
    }
}

save_dfs = function(list_dfs, save_prefix){
    for (name in names(list_dfs)){
        if (dim(list_dfs[[name]])[1]>0){
            save_filepath = paste0(save_prefix, '_', name,'.csv')
            write.csv(list_dfs[[name]], save_filepath)            
        }

    }
}

                            
# get list of unique genes in list of lists
get_genes_unique = function(list_dfs){
    genelist = sort(unique(do.call('c',lapply(do.call('rbind',list_dfs)$geneID, function(x) strsplit(x,'/')[[1]]))))
    return(genelist)
}
run_enrichment = function(entrez_list, c_gmt, save_prefix){
    enrich_df_list =  sapply(entrez_list, function(x) enrichr_res(x,c_gmt))
    save_dfs(enrich_df_list, save_prefix)
    enrich_df_list_filt = sapply(enrich_df_list, filt_res)
    get_lengths(enrich_df_list_filt)
    print('getting unique genes --all')
    print(get_genes_unique (enrich_df_list)                        )
    print('getting unique genes --pval filt')
    print(get_genes_unique (enrich_df_list_filt)   )
    return(enrich_df_list_filt)
}
 

In [147]:
dz_gene_list_sym = sapply(diseases,get_genes)

[1] "airway"
[1] 279
[1] "ast"
[1] 39
[1] "colon"
[1] 183
[1] "endo"
[1] 79
[1] "eso"
[1] 38
[1] "gm"
[1] 96
[1] "hmec"
[1] 436
[1] "kc"
[1] 51
[1] "mc"
[1] 60
[1] "ov"
[1] 74
[1] "panc"
[1] 30
[1] "pros"
[1] 151
[1] "renal"
[1] 9
[1] "thy"
[1] 21


In [148]:
dz_gene_list_entrez = sapply(dz_gene_list_sym,get_entrez)
get_lengths(dz_gene_list_entrez)

'select()' returned 1:1 mapping between keys and columns

Warning message in bitr(genes, fromType = "SYMBOL", toType = "ENTREZID", OrgDb = "org.Hs.eg.db"):
“13.26% of input gene IDs are fail to map...”
'select()' returned 1:1 mapping between keys and columns

Warning message in bitr(genes, fromType = "SYMBOL", toType = "ENTREZID", OrgDb = "org.Hs.eg.db"):
“12.82% of input gene IDs are fail to map...”
'select()' returned 1:1 mapping between keys and columns

Warning message in bitr(genes, fromType = "SYMBOL", toType = "ENTREZID", OrgDb = "org.Hs.eg.db"):
“6.56% of input gene IDs are fail to map...”
'select()' returned 1:1 mapping between keys and columns

Warning message in bitr(genes, fromType = "SYMBOL", toType = "ENTREZID", OrgDb = "org.Hs.eg.db"):
“8.86% of input gene IDs are fail to map...”
'select()' returned 1:1 mapping between keys and columns

Warning message in bitr(genes, fromType = "SYMBOL", toType = "ENTREZID", OrgDb = "org.Hs.eg.db"):
“10.53% of input gene IDs are fail to 

[1] "airway"
[1] 242
[1] "ast"
[1] 34
[1] "colon"
[1] 171
[1] "endo"
[1] 72
[1] "eso"
[1] 34
[1] "gm"
[1] 88
[1] "hmec"
[1] 384
[1] "kc"
[1] 46
[1] "mc"
[1] 54
[1] "ov"
[1] 60
[1] "panc"
[1] 29
[1] "pros"
[1] 139
[1] "renal"
[1] 8
[1] "thy"
[1] 19


# 1. enrichment of egenes drug target (from pt 1 of 7B)

In [150]:
save_repurpose_gmt = '/Users/mguo123/Google Drive/0_altman/db/cmap/repurposing_drugs_target.gmt'
# save_repurpose_gmt = '/Users/mguo123/Google Drive/0_altman/db/cmap/repurposing_drugs_target_cat.gmt'
c_repurpose <- read.gmt(save_repurpose_gmt)

dz_repurpose_drug_filt = run_enrichment(entrez_list=dz_gene_list_entrez,
                          c_gmt = c_repurpose,
                          save_prefix=paste0(save_path, 'direct_drug_target'))
dz_repurpose_drug_filt

--> No gene can be mapped....

--> Expected input gene ID: 2548,1586,2030,1586,3351,3315

--> return NULL...



[1] "airway"
[1] 20
[1] "ast"
[1] 1
[1] "colon"
[1] 14
[1] "endo"
[1] 8
[1] "eso"
[1] 3
[1] "gm"
[1] 2
[1] "hmec"
[1] 0
[1] "kc"
[1] 7
[1] "mc"
[1] 3
[1] "ov"
[1] 1
[1] "panc"
[1] 2
[1] "pros"
[1] 2
[1] "renal"
[1] 1
[1] "thy"
[1] 0
[1] "getting unique genes --all"
 [1] "AGER"     "AKT1"     "ATF4"     "BLK"      "CACNG6"   "CATSPER1"
 [7] "CCND1"    "CES1"     "CHRNA2"   "COL1A1"   "CRHR1"    "CTSF"    
[13] "CYP51A1"  "DDAH2"    "EGFR"     "FAAH"     "FADS1"    "FDPS"    
[19] "FGFR2"    "GABBR1"   "GGCX"     "GPBAR1"   "HAPLN4"   "HCN3"    
[25] "HSD17B1"  "INSR"     "KCNN4"    "LDHA"     "MAP1A"    "MAPT"    
[31] "MC1R"     "MMP2"     "NEU1"     "ORM2"     "PADI4"    "PADI6"   
[37] "PKD2"     "POMC"     "PPIF"     "PRKACA"   "PSMB8"    "PSMB9"   
[43] "PTK2B"    "RETSAT"   "SCN2B"    "SCNN1A"   "SLC7A1"   "STAT3"   
[49] "TNF"      "TUBB"     "TUBB3"    "TUBG1"    "VDR"     
[1] "getting unique genes --pval filt"
 [1] "AGER"    "CCND1"   "CHRNA2"  "CRHR1"   "EGFR"    "FADS1"   "F

ID,Description,GeneRatio,BgRatio,pvalue,p.adjust,qvalue,geneID,Count
<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<chr>,<int>
benfotiamine,benfotiamine,1/14,1/1131,0.01237843,0.04580018,0.02215087,AGER,1
ciprofloxacin,ciprofloxacin,1/14,1/1131,0.01237843,0.04580018,0.02215087,TNF,1
cisatracurium,cisatracurium,1/14,1/1131,0.01237843,0.04580018,0.02215087,CHRNA2,1
cobimetinib,cobimetinib,1/14,1/1131,0.01237843,0.04580018,0.02215087,MAP1A,1
doxorubicin,doxorubicin,1/14,1/1131,0.01237843,0.04580018,0.02215087,TNF,1
enoxacin,enoxacin,1/14,1/1131,0.01237843,0.04580018,0.02215087,TNF,1
etoposide-phosphate,etoposide-phosphate,1/14,1/1131,0.01237843,0.04580018,0.02215087,TNF,1
fleroxacin,fleroxacin,1/14,1/1131,0.01237843,0.04580018,0.02215087,TNF,1
garenoxacin,garenoxacin,1/14,1/1131,0.01237843,0.04580018,0.02215087,TNF,1


In [151]:
save_repurpose_gmt = '/Users/mguo123/Google Drive/0_altman/db/cmap/repurposing_drugs_target_cat.gmt'
c_repurpose <- read.gmt(save_repurpose_gmt)

dz_repurpose_drug_cat_filt = run_enrichment(entrez_list=dz_gene_list_entrez,
                          c_gmt = c_repurpose,
                          save_prefix=paste0(save_path, 'direct_drug_target_cat'))
dz_repurpose_drug_cat_filt                                


--> No gene can be mapped....

--> Expected input gene ID: 1346,1128,8647,3775,2554,4067

--> return NULL...



[1] "airway"
[1] 0
[1] "ast"
[1] 0
[1] "colon"
[1] 0
[1] "endo"
[1] 0
[1] "eso"
[1] 0
[1] "gm"
[1] 0
[1] "hmec"
[1] 0
[1] "kc"
[1] 0
[1] "mc"
[1] 0
[1] "ov"
[1] 0
[1] "panc"
[1] 0
[1] "pros"
[1] 0
[1] "renal"
[1] 0
[1] "thy"
[1] 0
[1] "getting unique genes --all"
 [1] "AGER"     "AKT1"     "ATF4"     "BLK"      "CACNG6"   "CATSPER1"
 [7] "CCND1"    "CES1"     "CHRNA2"   "COL1A1"   "CRHR1"    "CTSF"    
[13] "CYP51A1"  "DDAH2"    "EGFR"     "FAAH"     "FADS1"    "FDPS"    
[19] "FGFR2"    "GABBR1"   "GGCX"     "GPBAR1"   "HAPLN4"   "HCN3"    
[25] "HSD17B1"  "INSR"     "KCNN4"    "LDHA"     "MAP1A"    "MAPT"    
[31] "MC1R"     "NEU1"     "ORM2"     "PADI4"    "PADI6"    "PKD2"    
[37] "POMC"     "PPIF"     "PRKACA"   "PSMB8"    "PSMB9"    "RETSAT"  
[43] "SCN2B"    "SCNN1A"   "SLC7A1"   "STAT3"    "TNF"      "TUBB"    
[49] "TUBB3"    "TUBG1"    "VDR"     
[1] "getting unique genes --pval filt"
NULL


ID,Description,GeneRatio,BgRatio,pvalue,p.adjust,qvalue,geneID,Count
<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<chr>,<int>
ID,Description,GeneRatio,BgRatio,pvalue,p.adjust,qvalue,geneID,Count
<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<lgl>,<chr>,<int>
ID,Description,GeneRatio,BgRatio,pvalue,p.adjust,qvalue,geneID,Count
<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<chr>,<int>
ID,Description,GeneRatio,BgRatio,pvalue,p.adjust,qvalue,geneID,Count
<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<chr>,<int>
ID,Description,GeneRatio,BgRatio,pvalue,p.adjust,qvalue,geneID,Count
<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<chr>,<int>
ID,Description,GeneRatio,BgRatio,pvalue,p.adjust,qvalue,geneID,Count


In [152]:
save_cmap_gmt = '/Users/mguo123/Google Drive/0_altman/db/cmap/gene_set_library_up_crisp_drug.gmt'
c_cmap<- read.gmt(save_cmap_gmt)

dz_cmap_up_drug_filt = run_enrichment(entrez_list=dz_gene_list_entrez,
                          c_gmt = c_cmap,
                          save_prefix=paste0(save_path, 'cmap_up_drug'))
dz_cmap_up_drug_filt                                


--> No gene can be mapped....

--> Expected input gene ID: 2352,4321,11183,6284,1786,6554

--> return NULL...



[1] "airway"
[1] 0
[1] "ast"
[1] 2
[1] "colon"
[1] 0
[1] "endo"
[1] 0
[1] "eso"
[1] 0
[1] "gm"
[1] 1
[1] "hmec"
[1] 0
[1] "kc"
[1] 0
[1] "mc"
[1] 0
[1] "ov"
[1] 0
[1] "panc"
[1] 0
[1] "pros"
[1] 0
[1] "renal"
[1] 0
[1] "thy"
[1] 0
[1] "getting unique genes --all"
 [1] "AGER"    "AKT1"    "ATF4"    "BLK"     "CCND1"   "CES1"    "CHRNA2" 
 [8] "COL1A1"  "CRHR1"   "CTSF"    "CYP51A1" "DDAH2"   "EGFR"    "FAAH"   
[15] "FADS1"   "FDPS"    "FGFR2"   "GABBR1"  "GGCX"    "GPBAR1"  "HCN3"   
[22] "HSD17B1" "INSR"    "KCNN4"   "LDHA"    "MAP1A"   "MAPT"    "MC1R"   
[29] "MMP2"    "NEU1"    "PADI4"   "PKD2"    "POMC"    "PRKACA"  "PSMB9"  
[36] "PTK2B"   "RETSAT"  "SCN2B"   "SCNN1A"  "SLC7A1"  "STAT3"   "TUBB3"  
[43] "TUBG1"  
[1] "getting unique genes --pval filt"
[1] "DDAH2" "EGFR"  "NEU1"  "PSMB9"


ID,Description,GeneRatio,BgRatio,pvalue,p.adjust,qvalue,geneID,Count
<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<chr>,<int>
ID,Description,GeneRatio,BgRatio,pvalue,p.adjust,qvalue,geneID,Count
<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<lgl>,<chr>,<int>
antazoline,antazoline,1/1,37/919,0.04026115,0.04461371,NA,EGFR,1
phenylpropanolamine,phenylpropanolamine,1/1,41/919,0.04461371,0.04461371,NA,EGFR,1
ID,Description,GeneRatio,BgRatio,pvalue,p.adjust,qvalue,geneID,Count
<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<chr>,<int>
ID,Description,GeneRatio,BgRatio,pvalue,p.adjust,qvalue,geneID,Count
<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<chr>,<int>
ID,Description,GeneRatio,BgRatio,pvalue,p.adjust,qvalue,geneID,Count


In [153]:
save_cmap_gmt = '/Users/mguo123/Google Drive/0_altman/db/cmap/gene_set_library_dn_crisp_drug.gmt'
c_cmap<- read.gmt(save_cmap_gmt)

dz_cmap_dn_drug_filt = run_enrichment(entrez_list=dz_gene_list_entrez,
                          c_gmt = c_cmap,
                          save_prefix=paste0(save_path, 'cmap_dn_drug'))
dz_cmap_dn_drug_filt                                


--> No gene can be mapped....

--> Expected input gene ID: 3567,5565,112714,51305,3156,231

--> return NULL...

--> No gene can be mapped....

--> Expected input gene ID: 55800,3779,27115,7226,6579,23620

--> return NULL...



[1] "airway"
[1] 0
[1] "ast"
[1] 0
[1] "colon"
[1] 0
[1] "endo"
[1] 0
[1] "eso"
[1] 0
[1] "gm"
[1] 0
[1] "hmec"
[1] 0
[1] "kc"
[1] 0
[1] "mc"
[1] 0
[1] "ov"
[1] 0
[1] "panc"
[1] 0
[1] "pros"
[1] 0
[1] "renal"
[1] 0
[1] "thy"
[1] 0
[1] "getting unique genes --all"
 [1] "AKT1"   "ATF4"   "CCND1"  "CES1"   "CHRNA2" "CTSF"   "EGFR"   "FAAH"  
 [9] "FGFR2"  "GGCX"   "HCN3"   "INSR"   "MAP1A"  "MC1R"   "MMP2"   "ORM2"  
[17] "PADI6"  "PKD2"   "PRKACA" "PSMB9"  "PTK2B"  "RETSAT" "SCNN1A" "SLC7A1"
[25] "STAT3"  "TNF"    "TUBB3"  "TUBG1"  "VDR"   
[1] "getting unique genes --pval filt"
NULL


ID,Description,GeneRatio,BgRatio,pvalue,p.adjust,qvalue,geneID,Count
<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<chr>,<int>
ID,Description,GeneRatio,BgRatio,pvalue,p.adjust,qvalue,geneID,Count
<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<chr>,<int>
ID,Description,GeneRatio,BgRatio,pvalue,p.adjust,qvalue,geneID,Count
<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<chr>,<int>
ID,Description,GeneRatio,BgRatio,pvalue,p.adjust,qvalue,geneID,Count
<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<chr>,<int>
ID,Description,GeneRatio,BgRatio,pvalue,p.adjust,qvalue,geneID,Count
<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<chr>,<int>
ID,Description,GeneRatio,BgRatio,pvalue,p.adjust,qvalue,geneID,Count


### drugs from cmap have no enrichment up or down

In [154]:
save_cmap_gmt_cat = '/Users/mguo123/Google Drive/0_altman/db/cmap/gene_set_library_up_crisp_drug_cat.gmt'
c_cmap<- read.gmt(save_cmap_gmt_cat)

dz_cmap_up_drug_cat_filt = run_enrichment(entrez_list=dz_gene_list_entrez,
                          c_gmt = c_cmap,
                          save_prefix=paste0(save_path, 'cmap_up_drug_cat'))
dz_cmap_up_drug_cat_filt                                


--> No gene can be mapped....

--> Expected input gene ID: 5724,5143,2065,438,2194,7173

--> return NULL...



[1] "airway"
[1] 0
[1] "ast"
[1] 0
[1] "colon"
[1] 0
[1] "endo"
[1] 0
[1] "eso"
[1] 0
[1] "gm"
[1] 0
[1] "hmec"
[1] 0
[1] "kc"
[1] 0
[1] "mc"
[1] 0
[1] "ov"
[1] 0
[1] "panc"
[1] 0
[1] "pros"
[1] 0
[1] "renal"
[1] 0
[1] "thy"
[1] 0
[1] "getting unique genes --all"
 [1] "AGER"    "AKT1"    "ATF4"    "BLK"     "CCND1"   "CES1"    "CHRNA2" 
 [8] "COL1A1"  "CRHR1"   "CTSF"    "CYP51A1" "DDAH2"   "EGFR"    "FAAH"   
[15] "FADS1"   "FDPS"    "FGFR2"   "GABBR1"  "GGCX"    "GPBAR1"  "HCN3"   
[22] "HSD17B1" "INSR"    "KCNN4"   "LDHA"    "MAP1A"   "MAPT"    "MC1R"   
[29] "MMP2"    "NEU1"    "POMC"    "PRKACA"  "PSMB9"   "PTK2B"   "RETSAT" 
[36] "SCN2B"   "SCNN1A"  "SLC7A1"  "STAT3"   "TUBB3"   "TUBG1"  
[1] "getting unique genes --pval filt"
NULL


ID,Description,GeneRatio,BgRatio,pvalue,p.adjust,qvalue,geneID,Count
<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<chr>,<int>
ID,Description,GeneRatio,BgRatio,pvalue,p.adjust,qvalue,geneID,Count
<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<lgl>,<chr>,<int>
ID,Description,GeneRatio,BgRatio,pvalue,p.adjust,qvalue,geneID,Count
<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<chr>,<int>
ID,Description,GeneRatio,BgRatio,pvalue,p.adjust,qvalue,geneID,Count
<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<chr>,<int>
ID,Description,GeneRatio,BgRatio,pvalue,p.adjust,qvalue,geneID,Count
<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<chr>,<int>
ID,Description,GeneRatio,BgRatio,pvalue,p.adjust,qvalue,geneID,Count


In [155]:
save_cmap_gmt_cat = '/Users/mguo123/Google Drive/0_altman/db/cmap/gene_set_library_dn_crisp_drug_cat.gmt'
c_cmap<- read.gmt(save_cmap_gmt_cat)

dz_cmap_dn_drug_cat_filt = run_enrichment(entrez_list=dz_gene_list_entrez,
                          c_gmt = c_cmap,
                          save_prefix=paste0(save_path, 'cmap_dn_drug_cat'))
dz_cmap_dn_drug_cat_filt                                


--> No gene can be mapped....

--> Expected input gene ID: 53919,6750,2492,51174,59340,6283

--> return NULL...

--> No gene can be mapped....

--> Expected input gene ID: 8841,2352,112714,5149,1147,1725

--> return NULL...



[1] "airway"
[1] 0
[1] "ast"
[1] 0
[1] "colon"
[1] 0
[1] "endo"
[1] 0
[1] "eso"
[1] 0
[1] "gm"
[1] 0
[1] "hmec"
[1] 0
[1] "kc"
[1] 0
[1] "mc"
[1] 0
[1] "ov"
[1] 0
[1] "panc"
[1] 0
[1] "pros"
[1] 0
[1] "renal"
[1] 0
[1] "thy"
[1] 0
[1] "getting unique genes --all"
 [1] "AKT1"   "ATF4"   "CCND1"  "CES1"   "CHRNA2" "CTSF"   "EGFR"   "FAAH"  
 [9] "HCN3"   "INSR"   "MAP1A"  "MC1R"   "MMP2"   "ORM2"   "PADI6"  "PKD2"  
[17] "PSMB9"  "PTK2B"  "RETSAT" "SCNN1A" "SLC7A1" "STAT3"  "TNF"    "TUBG1" 
[25] "VDR"   
[1] "getting unique genes --pval filt"
NULL


ID,Description,GeneRatio,BgRatio,pvalue,p.adjust,qvalue,geneID,Count
<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<chr>,<int>
ID,Description,GeneRatio,BgRatio,pvalue,p.adjust,qvalue,geneID,Count
<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<chr>,<int>
ID,Description,GeneRatio,BgRatio,pvalue,p.adjust,qvalue,geneID,Count
<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<chr>,<int>
ID,Description,GeneRatio,BgRatio,pvalue,p.adjust,qvalue,geneID,Count
<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<chr>,<int>
ID,Description,GeneRatio,BgRatio,pvalue,p.adjust,qvalue,geneID,Count
<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<chr>,<int>
ID,Description,GeneRatio,BgRatio,pvalue,p.adjust,qvalue,geneID,Count


### drug categories from cmap have no enrichment up or down

interesting drug egenes

In [183]:
repurposing_info = read.csv('/Users/mguo123/Google Drive/0_altman/db/cmap/repurposing_drugs_20200324.txt',
                            stringsAsFactor=F,sep='\t',skip=9)
repurposing_info = repurposing_info%>%separate_rows(target,sep='[|]')
head(repurposing_info)

,pert_iname,clinical_phase,moa,target,disease_area,indication
,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
1,(R)-(-)-apomorphine,Launched,dopamine receptor agonist,ADRA2A,neurology/psychiatry,Parkinson's Disease
2,(R)-(-)-apomorphine,Launched,dopamine receptor agonist,ADRA2B,neurology/psychiatry,Parkinson's Disease
3,(R)-(-)-apomorphine,Launched,dopamine receptor agonist,ADRA2C,neurology/psychiatry,Parkinson's Disease
4,(R)-(-)-apomorphine,Launched,dopamine receptor agonist,CALY,neurology/psychiatry,Parkinson's Disease
5,(R)-(-)-apomorphine,Launched,dopamine receptor agonist,DRD1,neurology/psychiatry,Parkinson's Disease
6,(R)-(-)-apomorphine,Launched,dopamine receptor agonist,DRD2,neurology/psychiatry,Parkinson's Disease


In [191]:
drug_egenes = get_genes_unique (dz_repurpose_drug)
length(drug_egenes)
drug_egenes_str = paste0(drug_egenes,collapse='|')
drug_egenes_info = data.frame(target=drug_egenes)%>%
    left_join(repurposing_info,by="target")
dim(drug_egenes_info)
length(unique(drug_egenes_info$target))

[1] 53

Warning message:
“Column `target` joining factor and character vector, coercing into character vector”


[1] 396   6

[1] 53

In [190]:
write.csv(drug_egenes_info, paste0(save_path,'drug_egenes_info.csv'))